In [1]:
!pip install openpyxl

In [2]:
!pip install kiwipiepy

     ---------------------------------------- 0.0/79.5 MB ? eta -:--:--
     --------- ----------------------------- 18.9/79.5 MB 92.1 MB/s eta 0:00:01
     ----------------- --------------------- 36.4/79.5 MB 89.2 MB/s eta 0:00:01
     -------------------------- ------------ 53.7/79.5 MB 87.8 MB/s eta 0:00:01
     ------------------------------------- - 76.3/79.5 MB 92.2 MB/s eta 0:00:01
     --------------------------------------- 79.5/79.5 MB 83.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 86.0 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.22.1-py3-none-any.whl size=79578122 sha256=dc6e86896a1c6bec436458bf17f0421623efca68d46194a4cc1152c14e5fa541
  Stored in directory: c:\users\b4-23\appdata\local\pip\cache\wheels\c2\cd\69\f6ec7f41232d2e0aeb74cfc3ee09cb

  DEPRECATION: Building 'kiwipiepy_model' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'kiwipiepy_model'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [8]:
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment
from kiwipiepy import Kiwi # 대문자면 클래스
from collections import Counter #갯수 확인
import re #re : 정규표현식

src = r"naverkeyword.xlsx" # r = raw 
out = r"naverkeyword_result_v1.xlsx"

# ==== 키워드 합계 & 사이즈 ====

def find_col_by_header(ws, keywords, header_row = 1) : #ws에서 kewords라는 컬럼을 1번째 행에서 찾아오라는 함수 선언
    # keywords = ["연관키워드", "월간검색수 (PC)", "월간검색수 (MOBILE)", .... , "월평균노출 광고수"] -> 리스트
    for col in range(1, ws.max_column + 1) : # 1부터 9까지의 값을 구하려면, (1, 10)의 값을 넣어야 한다
        v = ws.cell(row=header_row, column=col).value # value = 값을 찾아라
        if not v : #만약에 중간에 값이 비어있다면
            continue #아래 명령을 따르지 말고 계속 해라. 비어있지 않다면 아래 명령을 따라라.
        s = str(v).replace(" ", "") 

        ok = True
        for kw in keywords :
            if kw.replace(" ", "") not in s :
                ok = False
                break
        if ok :
            return col 
    return None

wb = load_workbook(src)
ws = wb.active #ws = work sheet, 활성화된 시트를 열겠다

pc_col = (find_col_by_header(ws, ["월간검색수", "PC"]) or find_col_by_header(ws, ["월간검색수", "피씨"]) or 2)
mo_col = (find_col_by_header(ws, ["월간검색수", "모바일"]) or find_col_by_header(ws, ["월간검색수", "MOBILE"]) or 3)

total_col = 4
size_col = 5

ws.cell(row=1, column=total_col).value = "전체검색량(PC+모바일)"
ws.cell(row=1, column=size_col).value = "키워드규모(전체검색량)"

for c in (total_col, size_col) :
    cell = ws.cell(row=1, column=c)
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")

def to_int(x) :
    if x is None :
        return 0
    if isinstance(x, (int, float)) :
        return int(x)
    s = str(x).strip()
    if s in ("<10", "< 10") :
        return 0
    s = s.replace(",", "")
    try : #아래 구문을 시도해보고 오류면 0으로 값을 만들고, 위의 연산은 그대로 처리해 (try 없으면 위 연산 진행X)
        return int(float(s))
    except :
        return 0

# 실수 형태를 가진 문자(예: "1,439.34를 한번에 int()하면 에러 발생. 실수로 먼저 변경한 후 정수로=>int(float())
        
for r in range(2, ws.max_row + 1) :
    pc = to_int(ws.cell(row=r, column=pc_col).value)
    mo = to_int(ws.cell(row=r, column=mo_col).value)
    total = pc + mo
    ws.cell(row=r, column=total_col).value = total

    if total >= 3000 :
        size = "대형(3000+)"
    elif total >= 1000 :
        size = "중형(1000+)"
    else :
        size = "소형(<1000)"
        
    ws.cell(row=r, column=size_col).value = size

ws.auto_filter.ref = f"A1:{ws.cell(row=1, column=max(ws.max_column, size_col)).coordinate}"
# auto_filter : 필터링 삽입

# wb.save(out)
# print("완료 :", out)

# ==== 키워드 형태소 분석 ====

fixed_category = {
    "원터치": ("타입", "원터치"),
    "드롭": ("타입", "드롭"),
    "클립": ("타입", "클립"),
    "명품": ("브랜드", "명품"),
    "브랜드": ("브랜드", "브랜드"),
    "여자": ("성별", "여자"),
}

brands = {
    "샤넬", "스톤헨지", "스와브로스키", "티파니", "까르띠에", "불가리", "반클리프", "구찌"
}


def classify_token(token: str) :
    t = str(token).strip()

    if not t :
        return ("기타", "")

    if t in fixed_category :
        return fixed_category[t]

    if t in brands :
        return("브랜드", t)

    return ("기타", "")
        
if "키워드_빈도" in wb.sheetnames :
    del wb["키워드_빈도"] #"키워드_빈도" 시트가 이미 있다면 지워라
ws_kw = wb.create_sheet("키워드_빈도") #없다면, 만들어라

keyword_col = 1 #첫번째 시트의 1번 열
texts = [
    row[0] for row in ws.iter_rows(min_row=2, max_col=1, values_only=True) 
    if row and row[0]
] 
#iter_rows: 각 행별로 ()를 해라
# ["귀걸이" ... ] tuple("귀걸이"...) -> 둘 다 인덱스, iter, 반복분 기능을 가졌지만 tuple은 추가,삭제,편집 불가

all_text = " ".join(map(str, texts)) 
#map: 'texts'(리스트)를 'str'(문자)로 바꿔라 그리고 " " 한 칸 띄어진 상태로 join 붙여라

kiwi = Kiwi()
tokens = kiwi.tokenize(all_text) #'귀걸이', '진주귀걸이' 이런걸 토큰이라고 함. kiwi가 형태소분석 해줌

noun_tags = {"NNG", "NNP", "NNB", "NR"} 
noun_tokens = [t.form for t in tokens if t.tag in noun_tags]

# [{"tag": "N", "form": "귀걸이"}, {...}]
# NNG=Noun Noun General: 일반명사(귀걸이, 반지, 목걸이..)
# NNP=Noun Noun Proper: 고유명사(샤넬, 스와브로스키, 티파니..)
# NNB=Noun Noun Bound: 의존명사(~개, ~명, ~번, ~가지..)
# NR=Noun Numeral: 수사(하나, 둘, 열.. 수치화할 수 있는 명사)

# Kiwi는 14K, 18K, (실버)925 같은 숫자가 들어간 명사를 찾아오지 못한다. 이를 보완하기 위해 re (정규표현식)

regex_tokens = re.findall(r"[A-Za-z]*\d+[A-Za-z]+|\d+[A-Za-z]*", all_text) 
#r"" 로우 스트링: 본연의 값을 그대로 사용해라
# '\d' : 숫자, '\d+' : 1개 이상의 숫자, '[A-Za-z]*\d' : a-z 영문자가 나올 수도 있고, 안 나올 수도 있다 *=랜덤
# '\d[A-Za-z]+' : 1개 이상의 문자가 뒤에 붙어 있다
# '|' 또는

stopwords = {"엑스", "구매", "다음", "때문", "생각"} #필요없는 명사 삽입
valid_one = {"금", "은", "동", "링", "침"} #살릴 명사 삽입

combined = noun_tokens + regex_tokens

filtered = [t for t in combined if (len(str(t)) > 1 or t in valid_one) and t not in stopwords] 

count = Counter(filtered)

ws_kw.append(["키워드", "빈도", "카테고리", "세부범주"])
for cell in ws_kw[1] :
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")

for kw, freq in count.most_common() :
    cat, sub = classify_token(kw)
    ws_kw.append([kw, freq, cat, sub])

ws_kw.auto_filter.ref = f"A1:D{ws_kw.max_row}"

wb.save(out)
print("완료 : ", out)

완료 :  naverkeyword_result_v1.xlsx
